In [ ]:
#DO NOT LOOK AT THIS CODE
#It repeats everything to run this experiment because I was facing import issues in the main repo
#Essentially useless, ignore but keep for now

In [5]:
# taken from the EPIC paper section 5
import numpy as np
from functools import wraps
import time
import numpy as np
Reward = np.ndarray
import numpy as np

def softmax(arr: np.ndarray) -> np.ndarray:
  exp = np.exp(arr)
  norm = np.sum(exp, axis=-1)
  norm = np.reshape(norm, (*arr.shape[0:-1], 1))
  norm = np.repeat(norm, arr.shape[-1], axis=-1)
  return exp / norm
  
def timed(f):
  @wraps(f)
  def wrapped(*args, **kwargs):
    st = time.perf_counter()
    out = f(*args, **kwargs)
    et = time.perf_counter()
    print(f'{f.__name__} took {et-st:.4f}s')
    return out
  return wrapped

class Env():
  def __init__(
    self,
    n_s: int,
    n_a: int,
    discount: float,
    init_dist: np.ndarray,
    transition_dist: np.ndarray,
  ):
    self.n_s = n_s
    self.states = np.arange(n_s)
    self.n_a = n_a
    self.actions = np.arange(n_a)
    self.discount = discount
    self.init_dist = init_dist
    self.transition_dist = transition_dist

class RandomEnv(Env):
  def __init__(self, n_s: int = 128, n_a: int = 16, discount: int = 0.9,
               episodic: bool = False):
    """
      Create a random environment.
      n_s: number of states
      n_a: number of actions
      discount: discount factor
      episodic: whether the environment is episodic - in this case episodic
        means that there is a terminal (self-looping) state
    """
    # uniform init dist
    init_dist = np.ones(n_s) / n_s

    # sample iid Gaussians, then only keep the highest values, then softmax -> sparse
    # TODO we currently pick the largest values as being 1.8sigma above mean
    # this means that if we have larger n_s we'll be picking more values
    # at this stage which will make transitions more uniform again
    thresh = 1 if n_s < 50 else (1.5 if n_s < 100 else 1.8) #! bigly goodn't hacky shit, kinda works for 32, 64, and 128
    transition_dist = np.random.randn(n_s, n_a, n_s)
    transition_dist = np.where(transition_dist > thresh,
                         transition_dist, np.zeros_like(transition_dist)-20)
    transition_dist = softmax(transition_dist)

    self.episodic = episodic
    if episodic:
      self.terminal_state = np.random.randint(0, n_s)
      for a in range(n_a):
        for s_prime in range(n_s):
          prob = 1.0 if s_prime == self.terminal_state else 0.0
          transition_dist[self.terminal_state, a, s_prime] = prob

    super().__init__(n_s, n_a, discount, init_dist, transition_dist)

"""Illustrative rewards for gridworlds."""

import numpy as np
def epic_gridworlds():
    SPARSE_GOAL = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 1]])

    CENTER_GOAL = np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]])

    OBSTACLE_COURSE = np.array([[0, -1, -1], [0, 0, 0], [-1, -1, 4]])

    CLIFF_WALK = np.array([[0, -1, -1], [0, 0, 0], [-4, -4, 4]])

    MANHATTAN_FROM_GOAL = np.array([[4, 3, 2], [3, 2, 1], [2, 1, 0]])

    ZERO = np.zeros((3, 3))

    REWARDS = {
        # Equivalent rewards
        "sparse_goal": {"state_reward": SPARSE_GOAL, "potential": ZERO},
        "sparse_goal_shift": {"state_reward": SPARSE_GOAL + 1, "potential": ZERO},
        "sparse_goal_scale": {"state_reward": SPARSE_GOAL * 10, "potential": ZERO},
        "dense_goal": {"state_reward": SPARSE_GOAL, "potential": -MANHATTAN_FROM_GOAL},
        "antidense_goal": {"state_reward": SPARSE_GOAL, "potential": MANHATTAN_FROM_GOAL},
        # Non-equivalent rewards
        "transformed_goal": {
            # Shifted, rescaled and reshaped sparse goal.
            "state_reward": SPARSE_GOAL * 4 - 1,
            "potential": -MANHATTAN_FROM_GOAL * 3,
        },
        "center_goal": {
            # Goal is in center
            "state_reward": CENTER_GOAL,
            "potential": ZERO,
        },
        "dirt_path": {
            # Some minor penalties to avoid to reach goal.
            #
            # Optimal policy for this is optimal in `SPARSE_GOAL`, but not equivalent.
            # Think may come apart in some dynamics but not particularly intuitively.
            "state_reward": OBSTACLE_COURSE,
            "potential": ZERO,
        },
        "cliff_walk": {
            # Avoid cliff to reach goal. Same set of optimal policies as `obstacle_course` in
            # deterministic dynamics, but not equivalent.
            #
            # Optimal policy differs in sufficiently slippery gridworlds as want to stay on top line
            # to avoid chance of falling off cliff.
            "state_reward": CLIFF_WALK,
            "potential": ZERO,
        },
        "sparse_penalty": {
            # Negative of `sparse_goal`.
            "state_reward": -SPARSE_GOAL,
            "potential": ZERO,
        },
        "evaluating_rewards/Zero-v0": {
            # All zero reward function
            "state_reward": ZERO,
            "potential": ZERO,
        },
    }

    sparse_var = REWARDS["sparse_goal"]["state_reward"]
    dense_var = REWARDS["dense_goal"]["state_reward"]
    cliff_var = REWARDS["cliff_walk"]["state_reward"]
    path_var = REWARDS["dirt_path"]["state_reward"]

    cliff_var_f = cliff_var.flatten()
    dense_var_f = dense_var.flatten()
    sparse_var_f = sparse_var.flatten()
    path_var_f = path_var.flatten()

    sparse_reward = np.zeros((9,5,9)) + sparse_var_f[:,None,None]
    dense_reward = np.zeros((9,5,9)) + dense_var_f[:,None,None]
    path_reward = np.zeros((9,5,9)) + path_var_f[:,None,None]
    cliff_reward = np.zeros((9,5,9)) + cliff_var_f[:,None,None]

    r = {'sparse': sparse_reward,
        'dense': dense_reward,
        'path': path_reward,
        'cliff': cliff_reward}

    env = RandomEnv(n_s=9, n_a=5, discount=0.99) 
    return env, r


In [7]:
import numpy as np
from typing import Union
import torch
from einops import rearrange
Policy = np.ndarray 
def get_state_dist(env: Env): # prob of transitioning into state
  return np.ones(env.n_s) / env.n_s
  # logits = env.transition_dist.sum(axis=(0, 1))
  # sum = logits.sum()
  # return logits / sum

def get_action_dist(env: Env): # right now this is just uniform
  return np.ones(env.n_a) / env.n_a

def epic_canon(reward: Reward, env: Env) -> Reward:
  D_s = get_state_dist(env)
  D_a = get_action_dist(env)
  if type(reward) is torch.Tensor:
    D_s, D_a = torch.tensor(D_s), torch.tensor(D_a)
  S = D_s[:, None, None]
  A = D_a[None, :, None]
  S_prime = D_s[None, None, :]

  potential = (reward * A * S_prime).sum(axis=(1, 2))

  term1 = env.discount * potential[None, None, :]
  term2 = potential[:, None, None]
  term3 = env.discount * (reward * S * A * S_prime).sum()

  return reward + term1 - term2 - term3

def dard_canon(reward: Reward, env: Env) -> Reward:
  A = get_action_dist(env)

  potential = (env.transition_dist * reward).sum(axis=2)
  potential = (potential * A[None, :]).sum(axis=1)

  term1 = env.discount * potential[None, None, :]
  term2 = potential[:, None, None]
  
  joint_probs = ( # [s, s', S', A, S'']; p(S', S'' | s, s', A=A)
    A[None, None, None, :, None] * 
    rearrange(env.transition_dist, 's A Sp -> s 1 Sp A 1') *
    rearrange(env.transition_dist, 'sp A Sd -> 1 sp 1 A Sd')
  )
  r_given_probs = reward[None, None, ...] * joint_probs
  term3 = env.discount * r_given_probs.sum(axis=(2,3,4))[:,None,:]
  
  return reward + term1 - term2 - term3

#! Does not converge for norm_ord 1 or inf
# @timed
def minimal_canon(
    reward: Reward, env: Env, norm_ord: Union[int, float], max_iters=100000,
) -> Reward:
  r = torch.tensor(reward)
  # potential = torch.tensor(reward.mean(axis=(1, 2)), requires_grad=True)
  potential = torch.zeros(env.n_s, requires_grad=True)
  frozen_potential = torch.clone(potential) 
  
  optimizer = torch.optim.Adam([potential], lr=1e-2)
  for i in range(max_iters):
    optimizer.zero_grad()
    r_prime = r + env.discount * potential[None, None, :] - potential[:, None, None]
    loss = torch.norm(r_prime, norm_ord)
    loss.backward()
    optimizer.step()
    # convergence = small gradient or potential hasn't changed in a while
    if torch.norm(potential.grad, 2) < 1e-4: break
    if i%10000 == 0 and i != 0:
      if torch.isclose(potential, frozen_potential, rtol=1e-3, atol=1e-3).all():
        # print(i)
        break
      else: frozen_potential = torch.clone(potential)
    if i==max_iters-1: print("Didn't converge")
  return r_prime.detach().numpy()

canon_funcs = {
  'None': lambda r, _: r,
  'EPIC': epic_canon,
  'DARD': dard_canon,
  'Minimal': minimal_canon,
}

# computes either the norm, or returns 1 if ord==0
# which makes it useful in defining canon_and_norm (where norm==0 means don't normalize)
def norm_wrapper(reward: Reward, ord: Union[int, float]) -> float:
  if ord == 0: return 1
  return np.linalg.norm(reward.flatten(), ord)

norm_opts = [1, 2, float('inf'), 0]
# returns a dictionary of all the possible canonicalizations and normalizations
def canon_and_norm(reward: Reward, env: Env) -> dict[str, Reward]:
  can = {c_name: canon_funcs[c_name](reward, env)
         for c_name in ['None', 'EPIC', 'DARD']}
  norm = {f'{c_name}-{n_ord}': val / norm_wrapper(val, n_ord)
            for n_ord in norm_opts
            for c_name, val in can.items()}
  # add in minimal canon (which depends on the norm order so it needs different code)
  for n_ord in norm_opts:
    if n_ord != 2: continue #! REMOVE ME
    if n_ord == 0: continue
    min_can = canon_funcs['Minimal'](reward, env, n_ord)
    norm[f'Minimal-{n_ord}'] = min_can / norm_wrapper(min_can, n_ord)
  return norm


In [9]:
import numpy as np

def optimize(env: Env, reward: Reward, convergence_thresh=1e-5) -> Policy:
  state_vals = np.zeros(env.n_s)

  for _ in range(10000):
    cond_p = env.transition_dist * (reward + env.discount * state_vals[None, None, :])
    new_vals = cond_p.sum(axis=2).max(axis=1)
    diff = state_vals - new_vals
    state_vals = new_vals
    if np.linalg.norm(diff, 2) < convergence_thresh: break
  
  return cond_p.sum(axis=2).argmax(axis=1)

# Monte Carlo estimation
def policy_returns(
  rewards: list[Reward],
  policy: Policy,
  env: Env,
  discount_thresh=1e-5,
) -> list[float]:
  # beyond this point, the discounts get so heavy that it's not worth computing
  steps_per_episode = round(np.log(discount_thresh) / np.log(env.discount))

  num_rs = len(rewards)

  # 2D array, first dim is different reward funcs, second dim is samples
  return_vals = [[] for _ in range(num_rs)]

  for _ in range(env.n_a): # do multiple repetitions to ensure low variance
    for episode_i in range(env.n_s):
      # init state - we want to have one episode for each possible starting state
      s = episode_i
      episode_rewards = [[] for _ in range(num_rs)] # same dims as return_vals

      for _ in range(steps_per_episode):
        # # sample action from policy
        # a = np.random.choice(env.actions, p=policy[s])
        a = policy[s]

        # next state
        s_next = np.random.choice(env.states, p=env.transition_dist[s, a])
        for i, r in enumerate(rewards):
          episode_rewards[i].append(r[s, a, s_next])
        s = s_next
      
      # at the end we compute the discounted return return
      for r_i, r_values in enumerate(episode_rewards): # for each return func
        return_val = 0 # accumulator for the return
        for i, r in enumerate(r_values):
          if i == 0: gamma_i = 1.0
          else: gamma_i *= env.discount
          return_val += gamma_i * r
        return_vals[r_i].append(return_val)

  return [sum(rs) / len(rs) for rs in return_vals]
  
# wrapper for the function above - takes just one reward function
def policy_return(reward: Reward, *args, **kwargs) -> Reward:
  return policy_returns([reward], *args, **kwargs)[0]


In [12]:
import itertools
import numpy as np
import json
def handpicked_experiment(results_path: str):

  dist_opts = [1, 2, float('inf')]

  np.random.seed(42)

  env, rewards = epic_gridworlds()

  # experiment starts here
  results = {}
  for (r1_name, r1), (r2_name, r2) in itertools.combinations(rewards.items(), 2):
    r_names = f'{r1_name}-{r2_name}'
    print(r_names)
    results[r_names] = {}

    # canonicalizations and normalizations of R1 and R2
    can1 = canon_and_norm(r1, env)
    can2 = canon_and_norm(r2, env)

    # compute the distances for all combinations of canon, norm, and dist
    for cn_name, r1_val in can1.items():
      r2_val = can2[cn_name]
      for d_ord in dist_opts:
        results[r_names][f'{cn_name}-{d_ord}'] = np.linalg.norm(
          (r1_val - r2_val).flatten(), d_ord
        )

    # calculate the best and worst policies under R1 and R2
    pi_1 = optimize(env, r1) # best policy under R1
    pi_x = optimize(env, -r1) # worst policy under R1
    pi_2 = optimize(env, r2) # best policy under R2
    pi_y = optimize(env, -r2) # worst policy under R2

    # compute return values
    J_1_pi_1, J_2_pi_1 = policy_returns([r1, r2], pi_1, env)
    J_1_pi_2, J_2_pi_2 = policy_returns([r1, r2], pi_2, env)
    J_1_pi_x = policy_return(r1, pi_x, env)
    J_2_pi_y = policy_return(r2, pi_y, env)
    # compute regrets (normalizing over max-min)
    results[r_names]['regret1'] = (J_1_pi_1 - J_1_pi_2) / (J_1_pi_1 - J_1_pi_x)
    results[r_names]['regret2'] = (J_2_pi_2 - J_2_pi_1) / (J_2_pi_2 - J_2_pi_y)

  # save as JSON
  with open(results_path, 'w') as f:
    json.dump(results, f)


if __name__ == '__main__':
  results_path = 'results/handpicked_epic_gridworlds_new.json'
  handpicked_experiment(results_path)

sparse-dense
sparse-path
sparse-cliff
dense-path
dense-cliff
path-cliff


In [14]:
#This is a temporary notebook for handpicked_new and epic_girdworlds_new.py until imports are fixed in the repo
import pandas as pd
import json

slippery = False

results = None
path = f"results/handpicked_epic_gridworlds_new.json"
with open(path, 'r') as f:
  results = json.load(f)
df = pd.DataFrame(results).T
cols_to_drop = [col for col in df.columns if '0' in col or 'None' in col]
df = df.drop(cols_to_drop, axis=1)
df

,EPIC-1-1,EPIC-1-2,EPIC-1-inf,DARD-1-1,DARD-1-2,DARD-1-inf,EPIC-2-1,EPIC-2-2,EPIC-2-inf,DARD-2-1,...,EPIC-inf-2,EPIC-inf-inf,DARD-inf-1,DARD-inf-2,DARD-inf-inf,Minimal-2-1,Minimal-2-2,Minimal-2-inf,regret1,regret2
sparse-dense,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002584,-0.002591
sparse-path,0.500000,0.026352,0.001389,0.542596,0.030021,0.003640,6.324555,0.320364,0.017568,6.264817,...,2.371708,0.125000,41.967300,2.320334,0.217393,4.967565,0.320364,0.024836,-0.003894,0.041638
sparse-cliff,1.022727,0.055204,0.004040,1.014147,0.062180,0.007143,13.443116,0.735118,0.055696,12.427736,...,6.665285,0.589286,96.751558,6.777368,0.812197,12.679754,0.735122,0.078891,0.060386,0.109954
dense-path,0.500000,0.026352,0.001389,0.542596,0.030021,0.003640,6.324555,0.320364,0.017568,6.264817,...,2.371708,0.125000,41.967300,2.320334,0.217393,4.967565,0.320364,0.024836,0.018071,0.029211
dense-cliff,1.022727,0.055204,0.004040,1.014147,0.062180,0.007143,13.443116,0.735118,0.055696,12.427736,...,6.665285,0.589286,96.751558,6.777368,0.812197,12.679754,0.735122,0.078891,0.054593,0.117298
path-cliff,0.772727,0.042989,0.004040,0.653546,0.040422,0.003986,9.956283,0.534426,0.039931,8.974813,...,5.093513,0.464286,70.730825,5.246258,0.601111,9.391973,0.534430,0.056587,0.054938,0.058804


In [15]:
cols = [col for col in df.columns if '2-2' in col or 'regret' in col]
df[cols]

,EPIC-2-2,DARD-2-2,Minimal-2-2,regret1,regret2
sparse-dense,0.000000,0.000000,0.000000,0.002584,-0.002591
sparse-path,0.320364,0.337912,0.320364,-0.003894,0.041638
sparse-cliff,0.735118,0.766884,0.735122,0.060386,0.109954
dense-path,0.320364,0.337912,0.320364,0.018071,0.029211
dense-cliff,0.735118,0.766884,0.735122,0.054593,0.117298
path-cliff,0.534426,0.558413,0.534430,0.054938,0.058804
